In [ ]:
# | hide
# | default_exp ops.shape
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

# Operations: Reshaping

In [ ]:
# | exporti

import numpy as np
from tidygrad.ops import BaseOp

In [ ]:
# | export
class Stack(BaseOp):
    """Stack tensors along a new axis"""

    name_template = "stack({})"
    
    def __init__(self, tensors, axis=0, name=None):
        assert isinstance(tensors, (list, tuple))
        super().__init__(*tensors, name=name)
        self.axis = axis
        self.set_out(np.stack([t.data for t in self.args], axis=axis))
    
    def backward(self):
        self.check_backward()
        key = [ slice(None) ] * len(self.out.shape)
        for i, a in enumerate(self.args):
            key[self.axis] = i

            a.accum_grad(self.out.grad[tuple(key)])

In [ ]:
#| export
class Concat(BaseOp):
    """Concat tensors along an existing axis"""

    name_template = "concat({})"
    
    def __init__(self, tensors, axis=0, name=None):
        super().__init__(*tensors, name=name)
        self.axis = axis
        self.set_out(np.concatenate([t.data for t in self.args], axis=axis))
    
    def backward(self):
        self.check_backward()
        prev_len = 0
        key = [slice(None)] * len(self.out.shape)
        for a in self.args:
            start = prev_len                 # sum of all previous lenght in axis dimension
            prev_len = start + a.shape[self.axis]
            key[self.axis] = slice(start, prev_len)
            a.accum_grad(self.out.grad[tuple(key)])